---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [0]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

In [0]:
def date_sorter():
    #answer code
    df1 = df.str.extract(
        '((?:\d{1,2}[/, -])?(?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[/,.-]? )?(?:\d{1,2})(?:st|nd|rd|th)?[/,-]? ?\d{2,4})'
        )

    def month_converter(month):
        months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        return months.index(month[:3]) + 1

    for n in range(len(df1)):   
        t = df1[n]
        if(re.search(r'\d{1,2}/\d{1,2}/\d{2}', df1[n]) and len(df1[n])<9):        
            i = t.rfind('/')
            df1[n] = df1[n].replace(t[i+1:],'19'+t[i+1:])
        elif(re.search(r'\d{1,2}-\d{1,2}-\d{2}', df1[n]) and len(df1[n])<9):        
            i = t.rfind('-')
            df1[n] = df1[n].replace(t[i+1:],'19'+t[i+1]+t[i+2])
            df1[n] = df1[n].replace('-','/')
        elif(re.search(r'\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}', df1[n])):
            t = df1[n].replace(' ','/')
            i1 = t.find('/')
            i2 = t.rfind('/')       
            df1[n] = str(month_converter(t[i1+1:i2])) + '/' + t[:i1] + '/' + t[i2+1:]
        elif(re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.? \d{1,2},? \d{4}', df1[n])):
            t = df1[n].replace(' ','/')
            t = t.replace(',','')
            i1 = t.find('/')
            i2 = t.rfind('/')       
            df1[n] = str(month_converter(t[:i1])) + '/' + t[i1+1:i2] + '/' + t[i2+1:]    
        elif(re.search(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,? \d{4}', df1[n])):
            t = df1[n].replace(' ','/')
            t = t.replace(',','')
            t = t.replace('.','')
            i1 = t.find('/') 
            df1[n] = str(month_converter(t[:i1])) + '/01/' + t[i1+1:]        
        elif(re.search(r'(?:\d{1,2}/)?\d{4}', df1[n])):
            i1 = t.find('/')        
            if len(df1[n])==4:
                df1[n] = '01/01/' + t[i1+1:]
            elif len(df1[n])<=7:
                df1[n] = t[:i1] + '/01/' + t[i1+1:]
            else:
                df1[n] = '01/12/2019'
        else:
            df1[n] = '01/12/2019'

    dframe = df1.to_frame(name='date')
    dframe['date'] = pd.to_datetime(dframe.date)
    dframe = dframe.sort_values(by='date')    
    return pd.Series(dframe.index)

In [0]:
#full regex divided in multiple regex -- just for reference
df1 = df.str.extract('(\d{1,2}[/ -,.]\d{1,2}[/ -,.]\d{2,4})')
print(len(df1.dropna()))
df2 = df.str.extract('((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,/]? \d{1,2}[.,/]? \d{2,4})')
# print(df2[100:200])
print(len(df2.dropna()))
df3 = df.str.extract('((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\-\d{1,2}\-\d{2,4})')
print(len(df3.dropna()))
df3a = df.str.extract('(\d{1,2} (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,/]? \d{2,4})')
print(len(df3a.dropna()))
df4 = df.str.extract('((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,/]? \d{1,2}(?:st|nd|rd|th), \d{2,4})')
# print(df4[200:300])
print(len(df4.dropna()))
df5 = df.str.extract('((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{2,4})')
print(len(df5.dropna()))
df6 = df.str.extract('(\d{1,2}/\d{2,4})')
print(len(df6.dropna()))
df7 = df.str.extract('(\d{2,4})')
print(len(df7.dropna()))
